In [3]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords,PorterStemmer
from nltk.tokenize import casual_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import brown
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [5]:
dataset = pd.read_csv("all-data.csv",
                      encoding="ISO-8859-1",
                      names=["target","text"])  
X_data = dataset['text'].values
y_data = dataset['target'].values

In [6]:
data_dummy = dataset.copy()
X_dummy = data_dummy['text']
y_clean = data_dummy['target']

# Convert to Lower Case
X_dummy = X_dummy.str.lower()
# Replace Number
X_dummy = X_dummy.str.replace("(\d+)[^ ]*",' ')
# Replace URL
X_dummy = X_dummy.str.replace("(http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*",' ')
# Replace Tag
X_dummy = X_dummy.str.replace("@[^ ]*",' ')
# Replace ' to nothing
X_dummy = X_dummy.str.replace("'",'')
# Remove Punctuation
X_dummy = X_dummy.str.replace('[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]',' ')
# Replace 3+ Consecutive Letter with only 2
X_dummy = X_dummy.str.replace("(.)\\1\\1+",'\\1\\1')
# Remove Independent words with length 1
X_dummy = X_dummy.str.replace("(^| ).( |$)",' ')

X_dummy[0:10]

In [7]:
X_clean = []
for sentence in X_dummy:
    # sentence = str(TextBlob(sentence).correct())
    tokenized = casual_tokenize(sentence)
    new_tokenized = ""
    for word in tokenized:
        #Remove Stopword
        word = remove_stopwords(word)
        if word == '':
            continue
        #Stemming
        stemmer = SnowballStemmer('english')
        word = stemmer.stem(word) 
        new_tokenized += (word + ' ')
    X_clean.append(new_tokenized.lstrip())

X_clean[0]

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train,X_test,y_train,y_test = train_test_split(X_clean,y_clean,test_size=0.3,random_state=101)
print(np.array(X_train).shape)
print(np.array(X_test).shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range=(1,2))
vect.fit(X_clean)
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,BaseDiscreteNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier  
import numpy as np

model = LogisticRegression(n_jobs=-1,max_iter=1000)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)